<a href="https://colab.research.google.com/github/iskra3138/GluonTS/blob/master/DeepAR_YJ1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install mxnet and gluonts

In [0]:
pip install --upgrade mxnet==1.6 gluonts

     |████████████████████████████████| 68.7MB 56kB/s 
     |████████████████████████████████| 419kB 42.0MB/s 
     |████████████████████████████████| 7.4MB 17.3MB/s 
     |████████████████████████████████| 194kB 39.3MB/s 
  Created wheel for ujson: filename=ujson-1.35-cp36-cp36m-linux_x86_64.whl size=68027 sha256=9ba0301c70049e9cc49ce6fb20e1e9e8b9bc42bd2497dc486247e2dddf8e8adf
  Stored in directory: /root/.cache/pip/wheels/28/77/e4/0311145b9c2e2f01470e744855131f9e34d6919687550f87d1
Successfully built ujson
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


## Dataset Loading

In [0]:
# Third-party imports
%matplotlib inline
import mxnet as mx
from mxnet import gluon
#import numpy as np
#import pandas as pd
import matplotlib.pyplot as plt
import json
#import os

In [0]:
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
#from gluonts.dataset.util import to_pandas

In [0]:
from gluonts.dataset.common import ListDataset
from gluonts.dataset.field_names import FieldName

In [0]:
from gluonts.model.deepar import DeepAREstimator
from gluonts.model.deep_factor import DeepFactorEstimator
from gluonts.distribution.gaussian import GaussianOutput
from gluonts.distribution.student_t import StudentTOutput
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

## Experiment of DeepAR

## electricity

In [0]:
elec_dataset = get_dataset("electricity", regenerate=False)

saving time-series into /root/.mxnet/gluon-ts/datasets/electricity/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/electricity/test/data.json


In [0]:
print (len(elec_dataset.train))
print (len(elec_dataset.test))

321
2247


In [0]:
elec_dataset.metadata

MetaData(freq='1H', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='321')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=24)

In [0]:
elec_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='321')

In [0]:
car = [int(elec_dataset.metadata.feat_static_cat[0].cardinality)]

In [0]:
next(iter(elec_dataset.train)).keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
test_ds_24 = ListDataset([{FieldName.START: entry['start'],
                        FieldName.TARGET: entry['target'],
                        FieldName.FEAT_STATIC_CAT: entry['feat_static_cat'],
                        FieldName.ITEM_ID: entry['item_id'],
                        }
                       for entry in iter(elec_dataset.test) if len(entry['target']) == 21068
                       ],
                     freq=elec_dataset.metadata.freq)

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.trainer.html
deepar_trainer=Trainer(
    ctx = 'cpu',
    epochs=100, # (default: 100).
    batch_size=32,# (default: 32).
    num_batches_per_epoch=50, # (default: 50).
    learning_rate=0.001, # (default:  10^−3 ).
    learning_rate_decay_factor=0.5, # (default: 0.5).
    patience = 10, # (default: 10).
    minimum_learning_rate=5e-05, # (default:  5⋅10^−5 ).
    clip_gradient = 10.0, # (default: 10).
    weight_decay=1e-08, #  (default  10^−8 ).
    init='xavier', #  (default: “xavier”).
    hybridize=True, 
    )

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html
DA_estimator = DeepAREstimator(
    freq=elec_dataset.metadata.freq, 
    prediction_length=elec_dataset.metadata.prediction_length,
    trainer = deepar_trainer, # (default: Trainer())
    context_length=None, # (default: None, in which case context_length = prediction_length)
    num_layers=2, # (default: 2)
    num_cells=40, # (default: 40)
    cell_type='lstm', # (default: ‘lstm’)
    dropout_rate=0.1, # (default: 0.1)
    use_feat_dynamic_real=False, # (default: False)
    use_feat_static_cat=True, # (default: False)
    use_feat_static_real=False, # (default: False)
    cardinality=car, # (default: None)
    #embedding_dimension=[20], # (default: [min(50, (cat+1)//2) for cat in cardinality])
    distr_output = StudentTOutput(), # (default: StudentTOutput())
    scaling=True, # (default: true)
    lags_seq=None, # (default: None, in which case these are automatically determined based on freq)
    time_features=None, # (default: None, in which case these are automatically determined based on freq)
    num_parallel_samples=100, # (default: 100)
)

In [0]:
DA_predictor = DA_estimator.train(elec_dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


 20%|██        | 10/50 [01:24<05:36,  8.41s/it, epoch=2/100, avg_epoch_loss=-.253]

In [0]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=elec_dataset.test, 
    predictor=DA_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(elec_dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 2247/2247 [00:00<00:00, 33522.33it/s]


{
    "MSE": 2505295.267445863,
    "abs_error": 8900631.609058857,
    "abs_target_sum": 128632956.0,
    "abs_target_mean": 2385.272140631948,
    "seasonal_error": 189.4933819611677,
    "MASE": 0.7794081356898931,
    "MAPE": 0.09735360795793072,
    "sMAPE": 0.1093649028734422,
    "OWA": NaN,
    "MSIS": 9.66649531494702,
    "QuantileLoss[0.1]": 3628471.544925059,
    "Coverage[0.1]": 0.07749221183800607,
    "QuantileLoss[0.5]": 8900631.654795162,
    "Coverage[0.5]": 0.3847908322207389,
    "QuantileLoss[0.9]": 5747472.161535261,
    "Coverage[0.9]": 0.8237279335410164,
    "RMSE": 1582.812454918732,
    "NRMSE": 0.6635773033845043,
    "ND": 0.06919402216846246,
    "wQuantileLoss[0.1]": 0.02820794653063138,
    "wQuantileLoss[0.5]": 0.06919402252401914,
    "wQuantileLoss[0.9]": 0.04468117922700355,
    "mean_wQuantileLoss": 0.04736104942721803,
    "MAE_Coverage": 0.0713296741334129
}


In [0]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=test_ds_24, 
    predictor=DA_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(test_ds_24))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 321/321 [00:00<00:00, 6511.89it/s]


{
    "MSE": 545833.2759793291,
    "abs_error": 910718.1942095757,
    "abs_target_sum": 17268997.0,
    "abs_target_mean": 2241.56243509865,
    "seasonal_error": 189.64815247902152,
    "MASE": 0.6705488337357872,
    "MAPE": 0.08118493418615869,
    "sMAPE": 0.10300329117623283,
    "OWA": NaN,
    "MSIS": 7.8536199536625615,
    "QuantileLoss[0.1]": 395009.03624981944,
    "Coverage[0.1]": 0.09319833852544135,
    "QuantileLoss[0.5]": 910718.1928385925,
    "Coverage[0.5]": 0.4445742471443409,
    "QuantileLoss[0.9]": 563131.3881342098,
    "Coverage[0.9]": 0.8646157840083073,
    "RMSE": 738.8053031613465,
    "NRMSE": 0.3295938991450988,
    "ND": 0.05273717947889942,
    "wQuantileLoss[0.1]": 0.022873884120184828,
    "wQuantileLoss[0.5]": 0.052737179399509566,
    "wQuantileLoss[0.9]": 0.032609385949526185,
    "mean_wQuantileLoss": 0.036073483156406856,
    "MAE_Coverage": 0.03253721010730349
}


## exchange_rate

In [0]:
exchange_rate_dataset = get_dataset("exchange_rate", regenerate=False)

saving time-series into /root/.mxnet/gluon-ts/datasets/exchange_rate/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/exchange_rate/test/data.json


In [0]:
print (len(exchange_rate_dataset.train))
print (len(exchange_rate_dataset.test))

8
40


In [0]:
exchange_rate_dataset.metadata

MetaData(freq='1B', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='8')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=30)

In [0]:
exchange_rate_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='8')

In [0]:
car = [int(exchange_rate_dataset.metadata.feat_static_cat[0].cardinality)]

In [0]:
next(iter(exchange_rate_dataset.train)).keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html
DA_estimator = DeepAREstimator(
    freq=exchange_rate_dataset.metadata.freq, 
    prediction_length=exchange_rate_dataset.metadata.prediction_length,
    trainer = deepar_trainer, # (default: Trainer())
    context_length=None, # (default: None, in which case context_length = prediction_length)
    num_layers=2, # (default: 2)
    num_cells=40, # (default: 40)
    cell_type='lstm', # (default: ‘lstm’)
    dropout_rate=0.1, # (default: 0.1)
    use_feat_dynamic_real=False, # (default: False)
    use_feat_static_cat=True, # (default: False)
    use_feat_static_real=False, # (default: False)
    cardinality=car, # (default: None)
    #embedding_dimension=[20], # (default: [min(50, (cat+1)//2) for cat in cardinality])
    distr_output = StudentTOutput(), # (default: StudentTOutput())
    scaling=True, # (default: true)
    lags_seq=None, # (default: None, in which case these are automatically determined based on freq)
    time_features=None, # (default: None, in which case these are automatically determined based on freq)
    num_parallel_samples=100, # (default: 100)
)

In [0]:
DA_predictor = DA_estimator.train(exchange_rate_dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:08<00:00,  6.07it/s, epoch=100/100, avg_epoch_loss=-4.39]


In [0]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=exchange_rate_dataset.test, 
    predictor=DA_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(exchange_rate_dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 40/40 [00:00<00:00, 612.87it/s]


{
    "MSE": 0.00020185745454047804,
    "abs_error": 10.993979050777853,
    "abs_target_sum": 975.9766580164433,
    "abs_target_mean": 0.8133138816803692,
    "seasonal_error": 0.006515919403041841,
    "MASE": 2.469324913488132,
    "MAPE": 0.013480804579762298,
    "sMAPE": 0.013343858052518864,
    "OWA": NaN,
    "MSIS": 37.279023397911615,
    "QuantileLoss[0.1]": 4.870998585969209,
    "Coverage[0.1]": 0.24333333333333335,
    "QuantileLoss[0.5]": 10.993979051709175,
    "Coverage[0.5]": 0.5800000000000001,
    "QuantileLoss[0.9]": 4.811509300395846,
    "Coverage[0.9]": 0.8683333333333334,
    "RMSE": 0.01420765478678582,
    "NRMSE": 0.01746884580087544,
    "ND": 0.011264592201540772,
    "wQuantileLoss[0.1]": 0.0049908966018397775,
    "wQuantileLoss[0.5]": 0.01126459220249502,
    "wQuantileLoss[0.9]": 0.0049299430072176804,
    "mean_wQuantileLoss": 0.007061810603850826,
    "MAE_Coverage": 0.085
}


## m4_daily

In [0]:
m4_daily_dataset = get_dataset("m4_daily", regenerate=True)

saving time-series into /root/.mxnet/gluon-ts/datasets/m4_daily/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/m4_daily/test/data.json


In [0]:
print (len(m4_daily_dataset.train))
print (len(m4_daily_dataset.test))

4227
4227


In [0]:
m4_daily_dataset.metadata

MetaData(freq='D', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='4227')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=14)

In [0]:
m4_daily_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='4227')

In [0]:
car = [int(m4_daily_dataset.metadata.feat_static_cat[0].cardinality)]

In [0]:
next(iter(m4_daily_dataset.train)).keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html
DA_estimator = DeepAREstimator(
    freq=m4_daily_dataset.metadata.freq, 
    prediction_length=m4_daily_dataset.metadata.prediction_length,
    trainer = deepar_trainer, # (default: Trainer())
    context_length=None, # (default: None, in which case context_length = prediction_length)
    num_layers=2, # (default: 2)
    num_cells=40, # (default: 40)
    cell_type='lstm', # (default: ‘lstm’)
    dropout_rate=0.1, # (default: 0.1)
    use_feat_dynamic_real=False, # (default: False)
    use_feat_static_cat=True, # (default: False)
    use_feat_static_real=False, # (default: False)
    cardinality=car, # (default: None)
    #embedding_dimension=[20], # (default: [min(50, (cat+1)//2) for cat in cardinality])
    distr_output = StudentTOutput(), # (default: StudentTOutput())
    scaling=True, # (default: true)
    lags_seq=None, # (default: None, in which case these are automatically determined based on freq)
    time_features=None, # (default: None, in which case these are automatically determined based on freq)
    num_parallel_samples=100, # (default: 100)
)

In [0]:
DA_predictor = DA_estimator.train(m4_daily_dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:04<00:00, 11.71it/s, epoch=100/100, avg_epoch_loss=5.87]


In [0]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=m4_daily_dataset.test, 
    predictor=DA_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(m4_daily_dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 4227/4227 [00:00<00:00, 59198.18it/s]


{
    "MSE": 436845.0172430013,
    "abs_error": 13690204.931915283,
    "abs_target_sum": 383097653.5168457,
    "abs_target_mean": 6473.649895515986,
    "seasonal_error": 52.78075560259988,
    "MASE": 4.171115561491951,
    "MAPE": 0.05080571587327816,
    "sMAPE": 0.0376818238510894,
    "OWA": NaN,
    "MSIS": 64.47320681031154,
    "QuantileLoss[0.1]": 7357084.496485901,
    "Coverage[0.1]": 0.13422217716043125,
    "QuantileLoss[0.5]": 13690204.932357788,
    "Coverage[0.5]": 0.42870661394437237,
    "QuantileLoss[0.9]": 8224832.873834228,
    "Coverage[0.9]": 0.7754570955422619,
    "RMSE": 660.9425218905207,
    "NRMSE": 0.10209735351124358,
    "ND": 0.03573554890310309,
    "wQuantileLoss[0.1]": 0.019204201406476096,
    "wQuantileLoss[0.5]": 0.03573554890425816,
    "wQuantileLoss[0.9]": 0.0214692854376164,
    "mean_wQuantileLoss": 0.02546967858278355,
    "MAE_Coverage": 0.07668615589126565
}


## m4_hourly

In [0]:
m4_hourly_dataset = get_dataset("m4_hourly", regenerate=False)

saving time-series into /root/.mxnet/gluon-ts/datasets/m4_hourly/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/m4_hourly/test/data.json


In [0]:
print (len(m4_hourly_dataset.train))
print (len(m4_hourly_dataset.test))

414
414


In [0]:
m4_hourly_dataset.metadata

MetaData(freq='H', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='414')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=48)

In [0]:
m4_hourly_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='414')

In [0]:
car = [int(m4_hourly_dataset.metadata.feat_static_cat[0].cardinality)]

In [0]:
next(iter(m4_hourly_dataset.train)).keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html
DA_estimator = DeepAREstimator(
    freq=m4_hourly_dataset.metadata.freq, 
    prediction_length=m4_hourly_dataset.metadata.prediction_length,
    trainer = deepar_trainer, # (default: Trainer())
    context_length=None, # (default: None, in which case context_length = prediction_length)
    num_layers=2, # (default: 2)
    num_cells=40, # (default: 40)
    cell_type='lstm', # (default: ‘lstm’)
    dropout_rate=0.1, # (default: 0.1)
    use_feat_dynamic_real=False, # (default: False)
    use_feat_static_cat=True, # (default: False)
    use_feat_static_real=False, # (default: False)
    cardinality=car, # (default: None)
    #embedding_dimension=[20], # (default: [min(50, (cat+1)//2) for cat in cardinality])
    distr_output = StudentTOutput(), # (default: StudentTOutput())
    scaling=True, # (default: true)
    lags_seq=None, # (default: None, in which case these are automatically determined based on freq)
    time_features=None, # (default: None, in which case these are automatically determined based on freq)
    num_parallel_samples=100, # (default: 100)
)

In [0]:
DA_predictor = DA_estimator.train(m4_hourly_dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:08<00:00,  5.75it/s, epoch=100/100, avg_epoch_loss=3.21]


In [0]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=m4_hourly_dataset.test, 
    predictor=DA_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(m4_hourly_dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 414/414 [00:00<00:00, 7404.27it/s]


{
    "MSE": 4756232.727302222,
    "abs_error": 9141971.046258926,
    "abs_target_sum": 145558863.59960938,
    "abs_target_mean": 7324.822041043147,
    "seasonal_error": 336.9046924038302,
    "MASE": 2.7229756432481858,
    "MAPE": 0.14478938542164052,
    "sMAPE": 0.12356179884852575,
    "OWA": NaN,
    "MSIS": 28.007460949354602,
    "QuantileLoss[0.1]": 5664822.299430752,
    "Coverage[0.1]": 0.2975543478260868,
    "QuantileLoss[0.5]": 9141971.167381763,
    "Coverage[0.5]": 0.7462258454106276,
    "QuantileLoss[0.9]": 3604911.2483688346,
    "Coverage[0.9]": 0.9298510466988733,
    "RMSE": 2180.8788887286296,
    "NRMSE": 0.2977381397812151,
    "ND": 0.06280600727555735,
    "wQuantileLoss[0.1]": 0.03891774200032951,
    "wQuantileLoss[0.5]": 0.06280600810768007,
    "wQuantileLoss[0.9]": 0.024766002971037957,
    "mean_wQuantileLoss": 0.04216325102634918,
    "MAE_Coverage": 0.15787707997852926
}


## m4_monthly

In [0]:
m4_monthly_dataset = get_dataset("m4_monthly", regenerate=False)

saving time-series into /root/.mxnet/gluon-ts/datasets/m4_monthly/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/m4_monthly/test/data.json


In [0]:
print (len(m4_monthly_dataset.train))
print (len(m4_monthly_dataset.test))

48000
48000


In [0]:
m4_monthly_dataset.metadata

MetaData(freq='M', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='48000')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=18)

In [0]:
m4_monthly_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='48000')

In [0]:
car = [int(m4_monthly_dataset.metadata.feat_static_cat[0].cardinality)]

In [0]:
next(iter(m4_monthly_dataset.train)).keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html
DA_estimator = DeepAREstimator(
    freq=m4_monthly_dataset.metadata.freq, 
    prediction_length=m4_monthly_dataset.metadata.prediction_length,
    trainer = deepar_trainer, # (default: Trainer())
    context_length=None, # (default: None, in which case context_length = prediction_length)
    num_layers=2, # (default: 2)
    num_cells=40, # (default: 40)
    cell_type='lstm', # (default: ‘lstm’)
    dropout_rate=0.1, # (default: 0.1)
    use_feat_dynamic_real=False, # (default: False)
    use_feat_static_cat=True, # (default: False)
    use_feat_static_real=False, # (default: False)
    cardinality=car, # (default: None)
    #embedding_dimension=[20], # (default: [min(50, (cat+1)//2) for cat in cardinality])
    distr_output = StudentTOutput(), # (default: StudentTOutput())
    scaling=True, # (default: true)
    lags_seq=None, # (default: None, in which case these are automatically determined based on freq)
    time_features=None, # (default: None, in which case these are automatically determined based on freq)
    num_parallel_samples=100, # (default: 100)
)

In [0]:
DA_predictor = DA_estimator.train(m4_monthly_dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:04<00:00, 11.60it/s, epoch=76/100, avg_epoch_loss=6.29]


In [0]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=m4_monthly_dataset.test, 
    predictor=DA_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(m4_monthly_dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 48000/48000 [00:00<00:00, 283352.25it/s]


{
    "MSE": 2157442.9546686537,
    "abs_error": 572127565.1197662,
    "abs_target_sum": 4157157295.381714,
    "abs_target_mean": 4811.5246474325495,
    "seasonal_error": 616.9975625355751,
    "MASE": 1.3527781366618112,
    "MAPE": 0.1798225742924299,
    "sMAPE": 0.15109219854808956,
    "OWA": NaN,
    "MSIS": 32.923053697481755,
    "QuantileLoss[0.1]": 414568237.27763677,
    "Coverage[0.1]": 0.2976192129629433,
    "QuantileLoss[0.5]": 572127565.2610779,
    "Coverage[0.5]": 0.4273321759259116,
    "QuantileLoss[0.9]": 519152351.21203303,
    "Coverage[0.9]": 0.6211539351852019,
    "RMSE": 1468.8236635718577,
    "NRMSE": 0.30527198158605057,
    "ND": 0.1376247094992909,
    "wQuantileLoss[0.1]": 0.09972397189256962,
    "wQuantileLoss[0.5]": 0.1376247095332833,
    "wQuantileLoss[0.9]": 0.12488157515443832,
    "mean_wQuantileLoss": 0.12074341886009708,
    "MAE_Coverage": 0.18304436728394327
}


## m4_quarterly

In [0]:
m4_quarterly_dataset = get_dataset("m4_quarterly", regenerate=False)

saving time-series into /root/.mxnet/gluon-ts/datasets/m4_quarterly/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/m4_quarterly/test/data.json


In [0]:
print (len(m4_quarterly_dataset.train))
print (len(m4_quarterly_dataset.test))

24000
24000


In [0]:
m4_quarterly_dataset.metadata

MetaData(freq='3M', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='24000')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=8)

In [0]:
m4_quarterly_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='24000')

In [0]:
car = [int(m4_quarterly_dataset.metadata.feat_static_cat[0].cardinality)]

In [0]:
next(iter(m4_quarterly_dataset.train)).keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html
DA_estimator = DeepAREstimator(
    freq=m4_quarterly_dataset.metadata.freq, 
    prediction_length=m4_quarterly_dataset.metadata.prediction_length,
    trainer = deepar_trainer, # (default: Trainer())
    context_length=None, # (default: None, in which case context_length = prediction_length)
    num_layers=2, # (default: 2)
    num_cells=40, # (default: 40)
    cell_type='lstm', # (default: ‘lstm’)
    dropout_rate=0.1, # (default: 0.1)
    use_feat_dynamic_real=False, # (default: False)
    use_feat_static_cat=True, # (default: False)
    use_feat_static_real=False, # (default: False)
    cardinality=car, # (default: None)
    #embedding_dimension=[20], # (default: [min(50, (cat+1)//2) for cat in cardinality])
    distr_output = StudentTOutput(), # (default: StudentTOutput())
    scaling=True, # (default: true)
    lags_seq=None, # (default: None, in which case these are automatically determined based on freq)
    time_features=None, # (default: None, in which case these are automatically determined based on freq)
    num_parallel_samples=100, # (default: 100)
)

In [0]:
DA_predictor = DA_estimator.train(m4_quarterly_dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:02<00:00, 20.08it/s, epoch=90/100, avg_epoch_loss=7.14]


In [0]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=m4_quarterly_dataset.test, 
    predictor=DA_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(m4_quarterly_dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 24000/24000 [00:00<00:00, 210833.50it/s]


{
    "MSE": 2024737.0321240863,
    "abs_error": 121131627.5148468,
    "abs_target_sum": 1147074070.9031982,
    "abs_target_mean": 5974.344119287491,
    "seasonal_error": 473.4332698179725,
    "MASE": 1.4459162301216213,
    "MAPE": 0.13600648498916967,
    "sMAPE": 0.11233684017897869,
    "OWA": NaN,
    "MSIS": 21.220881088496718,
    "QuantileLoss[0.1]": 77955880.22958528,
    "Coverage[0.1]": 0.19675520833333332,
    "QuantileLoss[0.5]": 121131627.54579163,
    "Coverage[0.5]": 0.58840625,
    "QuantileLoss[0.9]": 79215289.81212462,
    "Coverage[0.9]": 0.854765625,
    "RMSE": 1422.9325465826153,
    "NRMSE": 0.23817385108247102,
    "ND": 0.1056005279759036,
    "wQuantileLoss[0.1]": 0.06796063323809887,
    "wQuantileLoss[0.5]": 0.10560052800288076,
    "wQuantileLoss[0.9]": 0.06905856545929161,
    "mean_wQuantileLoss": 0.08087324223342375,
    "MAE_Coverage": 0.07679861111111112
}


## m4_weekly

In [0]:
m4_weekly_dataset = get_dataset("m4_weekly", regenerate=False)

saving time-series into /root/.mxnet/gluon-ts/datasets/m4_weekly/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/m4_weekly/test/data.json


In [0]:
print (len(m4_weekly_dataset.train))
print (len(m4_weekly_dataset.test))

359
359


In [0]:
m4_weekly_dataset.metadata

MetaData(freq='W', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='359')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=13)

In [0]:
m4_weekly_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='359')

In [0]:
car = [int(m4_weekly_dataset.metadata.feat_static_cat[0].cardinality)]

In [0]:
next(iter(m4_weekly_dataset.train)).keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html
DA_estimator = DeepAREstimator(
    freq=m4_weekly_dataset.metadata.freq, 
    prediction_length=m4_weekly_dataset.metadata.prediction_length,
    trainer = deepar_trainer, # (default: Trainer())
    context_length=None, # (default: None, in which case context_length = prediction_length)
    num_layers=2, # (default: 2)
    num_cells=40, # (default: 40)
    cell_type='lstm', # (default: ‘lstm’)
    dropout_rate=0.1, # (default: 0.1)
    use_feat_dynamic_real=False, # (default: False)
    use_feat_static_cat=True, # (default: False)
    use_feat_static_real=False, # (default: False)
    cardinality=car, # (default: None)
    #embedding_dimension=[20], # (default: [min(50, (cat+1)//2) for cat in cardinality])
    distr_output = StudentTOutput(), # (default: StudentTOutput())
    scaling=True, # (default: true)
    lags_seq=None, # (default: None, in which case these are automatically determined based on freq)
    time_features=None, # (default: None, in which case these are automatically determined based on freq)
    num_parallel_samples=100, # (default: 100)
)

In [0]:
DA_predictor = DA_estimator.train(m4_weekly_dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:03<00:00, 14.95it/s, epoch=100/100, avg_epoch_loss=6.29]


In [0]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=m4_weekly_dataset.test, 
    predictor=DA_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(m4_weekly_dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 359/359 [00:00<00:00, 5296.21it/s]


{
    "MSE": 384410.79926126904,
    "abs_error": 1590894.3291549683,
    "abs_target_sum": 25616889.502441406,
    "abs_target_mean": 5488.941397566194,
    "seasonal_error": 216.54813922503882,
    "MASE": 3.3199411936941767,
    "MAPE": 0.08436536268062009,
    "sMAPE": 0.08431301616954852,
    "OWA": NaN,
    "MSIS": 39.34233149181943,
    "QuantileLoss[0.1]": 801268.6149017336,
    "Coverage[0.1]": 0.21277051639168668,
    "QuantileLoss[0.5]": 1590894.314628601,
    "Coverage[0.5]": 0.6164559674308977,
    "QuantileLoss[0.9]": 771020.7943069458,
    "Coverage[0.9]": 0.9346475251767725,
    "RMSE": 620.0087090205017,
    "NRMSE": 0.11295597167341132,
    "ND": 0.06210333729250574,
    "wQuantileLoss[0.1]": 0.031278919121908576,
    "wQuantileLoss[0.5]": 0.062103336725443645,
    "wQuantileLoss[0.9]": 0.03009814264270703,
    "mean_wQuantileLoss": 0.04116013283001975,
    "MAE_Coverage": 0.08795800299978562
}


## m4_yearly

In [0]:
m4_yearly_dataset = get_dataset("m4_yearly", regenerate=False)

saving time-series into /root/.mxnet/gluon-ts/datasets/m4_yearly/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/m4_yearly/test/data.json


In [0]:
print (len(m4_yearly_dataset.train))
print (len(m4_yearly_dataset.test))

23000
23000


In [0]:
m4_yearly_dataset.metadata

MetaData(freq='12M', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='23000')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=6)

In [0]:
m4_yearly_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='23000')

In [0]:
car = [int(m4_yearly_dataset.metadata.feat_static_cat[0].cardinality)]

In [0]:
next(iter(m4_yearly_dataset.train)).keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html
DA_estimator = DeepAREstimator(
    freq=m4_yearly_dataset.metadata.freq, 
    prediction_length=m4_yearly_dataset.metadata.prediction_length,
    trainer = deepar_trainer, # (default: Trainer())
    context_length=None, # (default: None, in which case context_length = prediction_length)
    num_layers=2, # (default: 2)
    num_cells=40, # (default: 40)
    cell_type='lstm', # (default: ‘lstm’)
    dropout_rate=0.1, # (default: 0.1)
    use_feat_dynamic_real=False, # (default: False)
    use_feat_static_cat=True, # (default: False)
    use_feat_static_real=False, # (default: False)
    cardinality=car, # (default: None)
    #embedding_dimension=[20], # (default: [min(50, (cat+1)//2) for cat in cardinality])
    distr_output = StudentTOutput(), # (default: StudentTOutput())
    scaling=True, # (default: true)
    lags_seq=None, # (default: None, in which case these are automatically determined based on freq)
    time_features=None, # (default: None, in which case these are automatically determined based on freq)
    num_parallel_samples=100, # (default: 100)
)

In [0]:
DA_predictor = DA_estimator.train(m4_yearly_dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:02<00:00, 21.45it/s, epoch=85/100, avg_epoch_loss=7.4]


In [0]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=m4_yearly_dataset.test, 
    predictor=DA_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(m4_yearly_dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 23000/23000 [00:00<00:00, 43603.97it/s]


{
    "MSE": 3919126.69023554,
    "abs_error": 145752912.43978882,
    "abs_target_sum": 860724679.6655273,
    "abs_target_mean": 6237.135359895143,
    "seasonal_error": 288.79964609563865,
    "MASE": 5.245301988194033,
    "MAPE": 0.2451058610301135,
    "sMAPE": 0.18763930514534188,
    "OWA": NaN,
    "MSIS": 103.04987363804142,
    "QuantileLoss[0.1]": 85261746.33708191,
    "Coverage[0.1]": 0.41618115942028944,
    "QuantileLoss[0.5]": 145752912.40400696,
    "Coverage[0.5]": 0.6689275362318828,
    "QuantileLoss[0.9]": 105988470.24346924,
    "Coverage[0.9]": 0.8192246376811587,
    "RMSE": 1979.6784310174064,
    "NRMSE": 0.31740187069640385,
    "ND": 0.16933743842041055,
    "wQuantileLoss[0.1]": 0.09905809412855936,
    "wQuantileLoss[0.5]": 0.16933743837883874,
    "wQuantileLoss[0.9]": 0.12313864438586303,
    "mean_wQuantileLoss": 0.13051139229775371,
    "MAE_Coverage": 0.1886280193236712
}


## traffic

In [0]:
traffic_dataset = get_dataset("traffic", regenerate=False)

saving time-series into /root/.mxnet/gluon-ts/datasets/traffic/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/traffic/test/data.json


In [0]:
print (len(traffic_dataset.train))
print (len(traffic_dataset.test))

862
6034


In [0]:
traffic_dataset.metadata

MetaData(freq='H', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='862')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=24)

In [0]:
traffic_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='862')

In [0]:
car = [int(traffic_dataset.metadata.feat_static_cat[0].cardinality)]

In [0]:
next(iter(traffic_dataset.train)).keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html
DA_estimator = DeepAREstimator(
    freq=traffic_dataset.metadata.freq, 
    prediction_length=traffic_dataset.metadata.prediction_length,
    trainer = deepar_trainer, # (default: Trainer())
    context_length=None, # (default: None, in which case context_length = prediction_length)
    num_layers=2, # (default: 2)
    num_cells=40, # (default: 40)
    cell_type='lstm', # (default: ‘lstm’)
    dropout_rate=0.1, # (default: 0.1)
    use_feat_dynamic_real=False, # (default: False)
    use_feat_static_cat=True, # (default: False)
    use_feat_static_real=False, # (default: False)
    cardinality=car, # (default: None)
    #embedding_dimension=[20], # (default: [min(50, (cat+1)//2) for cat in cardinality])
    distr_output = StudentTOutput(), # (default: StudentTOutput())
    scaling=True, # (default: true)
    lags_seq=None, # (default: None, in which case these are automatically determined based on freq)
    time_features=None, # (default: None, in which case these are automatically determined based on freq)
    num_parallel_samples=100, # (default: 100)
)

In [0]:
DA_predictor = DA_estimator.train(traffic_dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:11<00:00,  4.30it/s, epoch=100/100, avg_epoch_loss=-3.96]


In [0]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=traffic_dataset.test, 
    predictor=DA_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(traffic_dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 6034/6034 [00:00<00:00, 51233.61it/s]


{
    "MSE": 0.0006300274364436295,
    "abs_error": 1352.253042393364,
    "abs_target_sum": 8672.5710073933,
    "abs_target_mean": 0.05988682885450042,
    "seasonal_error": 0.015220711169889612,
    "MASE": 0.5983531101197326,
    "MAPE": 0.23569191746745768,
    "sMAPE": 0.16275750719598817,
    "OWA": NaN,
    "MSIS": 8.844226183430925,
    "QuantileLoss[0.1]": 637.7551666090624,
    "Coverage[0.1]": 0.1023022317975912,
    "QuantileLoss[0.5]": 1352.2530407310182,
    "Coverage[0.5]": 0.5676375538614535,
    "QuantileLoss[0.9]": 919.033253759315,
    "Coverage[0.9]": 0.9137802452767645,
    "RMSE": 0.025100347337111285,
    "NRMSE": 0.41912967871607426,
    "ND": 0.15592297154333803,
    "wQuantileLoss[0.1]": 0.07353703602603899,
    "wQuantileLoss[0.5]": 0.15592297135165947,
    "wQuantileLoss[0.9]": 0.10597010424888377,
    "mean_wQuantileLoss": 0.11181003720886074,
    "MAE_Coverage": 0.027906676978603065
}


## electricity2

In [0]:
elec_dataset = get_dataset("electricity", regenerate=False)

In [0]:
print (len(elec_dataset.train))
print (len(elec_dataset.test))

321
2247


In [0]:
elec_dataset.metadata

MetaData(freq='1H', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='321')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=24)

In [0]:
elec_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='321')

In [0]:
car = [int(elec_dataset.metadata.feat_static_cat[0].cardinality)]

In [0]:
next(iter(elec_dataset.train)).keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
test_ds_24 = ListDataset([{FieldName.START: entry['start'],
                        FieldName.TARGET: entry['target'],
                        FieldName.FEAT_STATIC_CAT: entry['feat_static_cat'],
                        FieldName.ITEM_ID: entry['item_id'],
                        }
                       for entry in iter(elec_dataset.test) if len(entry['target']) == 21068
                       ],
                     freq=elec_dataset.metadata.freq)

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.trainer.html
deepar_trainer=Trainer(
    ctx = 'cpu',
    epochs=100, # (default: 100).
    batch_size=32,# (default: 32).
    num_batches_per_epoch=50, # (default: 50).
    learning_rate=0.001, # (default:  10^−3 ).
    learning_rate_decay_factor=0.5, # (default: 0.5).
    patience = 10, # (default: 10).
    minimum_learning_rate=5e-05, # (default:  5⋅10^−5 ).
    clip_gradient = 10.0, # (default: 10).
    weight_decay=1e-08, #  (default  10^−8 ).
    init='xavier', #  (default: “xavier”).
    hybridize=True, 
    )

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html
DA_estimator = DeepAREstimator(
    freq=elec_dataset.metadata.freq, 
    prediction_length=elec_dataset.metadata.prediction_length,
    trainer = deepar_trainer, # (default: Trainer())
    context_length=None, # (default: None, in which case context_length = prediction_length)
    num_layers=2, # (default: 2)
    num_cells=40, # (default: 40)
    cell_type='lstm', # (default: ‘lstm’)
    dropout_rate=0.1, # (default: 0.1)
    use_feat_dynamic_real=False, # (default: False)
    use_feat_static_cat=True, # (default: False)
    use_feat_static_real=False, # (default: False)
    cardinality=car, # (default: None)
    #embedding_dimension=[20], # (default: [min(50, (cat+1)//2) for cat in cardinality])
    distr_output = StudentTOutput(), # (default: StudentTOutput())
    scaling=True, # (default: true)
    lags_seq=None, # (default: None, in which case these are automatically determined based on freq)
    time_features=None, # (default: None, in which case these are automatically determined based on freq)
    num_parallel_samples=100, # (default: 100)
)

In [0]:
DA_predictor = DA_estimator.train(elec_dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:15<00:00,  3.31it/s, epoch=100/100, avg_epoch_loss=5.13]


In [0]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=elec_dataset.test, 
    predictor=DA_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(elec_dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 2247/2247 [00:00<00:00, 22936.20it/s]


{
    "MSE": 1896246.6231927625,
    "abs_error": 8258335.360194206,
    "abs_target_sum": 128632956.0,
    "abs_target_mean": 2385.272140631948,
    "seasonal_error": 189.4933819611677,
    "MASE": 0.718082166845792,
    "MAPE": 0.09625337721242021,
    "sMAPE": 0.10614491706587359,
    "OWA": NaN,
    "MSIS": 8.2256149940051,
    "QuantileLoss[0.1]": 3532141.34392297,
    "Coverage[0.1]": 0.10074543836226056,
    "QuantileLoss[0.5]": 8258335.3574573975,
    "Coverage[0.5]": 0.500259605399792,
    "QuantileLoss[0.9]": 4943708.275373017,
    "Coverage[0.9]": 0.8834000890075647,
    "RMSE": 1377.0427092841974,
    "NRMSE": 0.5773105239552948,
    "ND": 0.06420077417947392,
    "wQuantileLoss[0.1]": 0.02745907000631292,
    "wQuantileLoss[0.5]": 0.06420077415819782,
    "wQuantileLoss[0.9]": 0.03843267253667884,
    "mean_wQuantileLoss": 0.04336417223372987,
    "MAE_Coverage": 0.005868318251495942
}


In [0]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=test_ds_24, 
    predictor=DA_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(test_ds_24))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 321/321 [00:00<00:00, 3768.33it/s]


{
    "MSE": 403157.531250229,
    "abs_error": 882678.3113889694,
    "abs_target_sum": 17268997.0,
    "abs_target_mean": 2241.56243509865,
    "seasonal_error": 189.64815247902152,
    "MASE": 0.6558522495129743,
    "MAPE": 0.08144418124880037,
    "sMAPE": 0.10157248045774643,
    "OWA": NaN,
    "MSIS": 7.048121356088902,
    "QuantileLoss[0.1]": 406979.5417923697,
    "Coverage[0.1]": 0.11240913811007272,
    "QuantileLoss[0.5]": 882678.320595948,
    "Coverage[0.5]": 0.5503634475597089,
    "QuantileLoss[0.9]": 479624.1469676733,
    "Coverage[0.9]": 0.905633437175493,
    "RMSE": 634.946872777738,
    "NRMSE": 0.2832608464683672,
    "ND": 0.05111346717988135,
    "wQuantileLoss[0.1]": 0.023567063089556948,
    "wQuantileLoss[0.5]": 0.05111346771303209,
    "wQuantileLoss[0.9]": 0.02777371187033464,
    "mean_wQuantileLoss": 0.03415141422430789,
    "MAE_Coverage": 0.022802007615091546
}


In [0]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=test_ds_24, 
    predictor=DA_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(test_ds_24))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 321/321 [00:00<00:00, 4050.65it/s]


{
    "MSE": 402712.0784618005,
    "abs_error": 883284.504096508,
    "abs_target_sum": 17268997.0,
    "abs_target_mean": 2241.56243509865,
    "seasonal_error": 189.64815247902152,
    "MASE": 0.6542245811034637,
    "MAPE": 0.08104167308167347,
    "sMAPE": 0.10131083651188916,
    "OWA": NaN,
    "MSIS": 7.0607845632515565,
    "QuantileLoss[0.1]": 409471.91812172765,
    "Coverage[0.1]": 0.11188992731048812,
    "QuantileLoss[0.5]": 883284.501845397,
    "Coverage[0.5]": 0.5571131879543093,
    "QuantileLoss[0.9]": 487177.15904294245,
    "Coverage[0.9]": 0.9088785046728972,
    "RMSE": 634.5959962541526,
    "NRMSE": 0.28310431434680267,
    "ND": 0.05114857012810345,
    "wQuantileLoss[0.1]": 0.023711389730493766,
    "wQuantileLoss[0.5]": 0.05114856999774782,
    "wQuantileLoss[0.9]": 0.02821108597349009,
    "mean_wQuantileLoss": 0.034357015233910555,
    "MAE_Coverage": 0.025960539979231537
}
